# Setup

In [26]:
import sys
import math
import time
import numpy as np
import pandas as pd
from pandas import DataFrame
from typing import Dict, Tuple, List
from datetime import datetime
from tqdm import tqdm
import json
import importlib

In [27]:
import src.models
importlib.reload(src.models)
from src.models import create_model
from src.evaluate import evaluate
importlib.reload(src.evaluate)

import src.conversation
importlib.reload(src.conversation)
from src.conversation import (
    setup_conversation,
    append_messages,
    remove_system_messages
)

## Functions

In [28]:
def process_results(results:List[Dict]) -> DataFrame:
    unprocessed_results_df = pd.DataFrame(results)

    # most common action
    grouped = unprocessed_results_df.groupby(['scenario_id', 'decision']).size().reset_index(name='counts')
    results_df = grouped.loc[grouped.groupby('scenario_id')['counts'].idxmax()]

    results_df['other_action'] = results_df['decision'].apply(lambda x: 'action1' if x == 'action2' else 'action2')

    # action_likelihood
    total_counts = unprocessed_results_df['scenario_id'].value_counts().reset_index()
    total_counts.columns = ['scenario_id', 'total_counts']

    results_df = results_df.merge(total_counts, on='scenario_id')
    results_df['answer_likelihood'] = results_df['counts'] / results_df['total_counts']

    results_df.rename(columns={'answer_likelihood':'action_likelihood'}, inplace=True)
    results_df = results_df[['scenario_id', 'decision', 'other_action', 'action_likelihood']]

    # action_entropy
    results_df['action_entropy'] = -((results_df['action_likelihood'] * np.log(results_df['action_likelihood'] + 1e-10)) + ((1 - results_df['action_likelihood']) * np.log(1 - results_df['action_likelihood'] + 1e-10)))

    unique_values = unprocessed_results_df[['scenario_id', 'model_id', 'eval_technique', 'eval_top_p', 'eval_temperature']].drop_duplicates()
    results_df = results_df.merge(unique_values, on='scenario_id')

    return results_df

# Playground

## Setup

In [29]:
num_scenarios = 680

target = create_model('anthropic/claude-3-haiku')
target_name = target.get_model_id().split("/")[-1]

dataset = "high"

if target._company == 'anthropic':
    system_msg = False
else:
    system_msg = True

turns = 4

In [30]:
scenarios = pd.read_csv(f"data/scenarios/moralchoice_{dataset}_ambiguity.csv")
scenarios_batch = scenarios[:num_scenarios]
scenarios_batch.shape

(680, 27)

In [31]:
max_tokens = 200      # default = 200
eval_temp = 1.0       # default = 1.0
eval_top_p = 1.0      # default = 1.0
eval_nb_samples = 2   # default = 1

## Initial eval

In [32]:
# COLLECT

initial_results = []
messages = []

for k, (identifier, scenario) in tqdm(
enumerate(scenarios_batch.iterrows()),
total=len(scenarios_batch),
position=0,
ncols=100,
leave=True,
):

    initial_results_i = evaluate(
        scenario,
        target,
        eval_nb_samples,
        max_tokens,
        eval_temp,
        eval_top_p,
        system_msg=system_msg,
    )

    # print(initial_results_i)
    initial_results.append(initial_results_i)

# PROCESS

flat_initial_results = [item for sublist in initial_results for item in sublist]
initial_results_unprocessed = pd.DataFrame(flat_initial_results)
initial_results_df = process_results(flat_initial_results)
initial_results_df.head()

initial_results_df.to_csv(f"results/initial/high{num_scenarios}_{target_name}.csv")

  1%|▉                                                           | 10/680 [02:07<2:54:26, 15.62s/it]

In [16]:
initial_results_df = initial_results_df.head(num_scenarios)

In [17]:
initial_results_df.shape

(5, 9)